In [46]:
import torch
import torch.nn.functional as F
from torchvision import transforms, models
from PIL import Image
import numpy as np

transform_pipeline = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


In [47]:

def load_model(checkpoint_path):
    model = models.resnet18(pretrained=False)  # Example model, change according to your needs
    # model.load_state_dict(torch.load(checkpoint_path))
    model.eval()
    return model

def load_image(image_path):
    image = Image.open(image_path).convert('RGB')
    # random image tensor
    image = torch.rand(3, 256, 256)
    return transform_pipeline(image).unsqueeze(0)

def process_activation(model, input_tensor, target_layer, iterations=30, lr=1.0):
    layer_output = {}

    def hook(module, input, output):
        layer_output[target_layer] = output

    handle = getattr(model, target_layer).register_forward_hook(hook)
    input_tensor.requires_grad = True

    for i in range(iterations):
        model.zero_grad()
        _ = model(input_tensor)
        activation = layer_output[target_layer].mean()
        activation.backward()
        input_tensor.data += lr * input_tensor.grad.data
        input_tensor.grad.zero_()

    handle.remove()
    return input_tensor.detach()

def deprocess_image(tensor):
    tensor = tensor * torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1) + torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)
    tensor = tensor.clamp(0, 1)
    tensor = tensor.numpy().transpose(1, 2, 0)
    return (tensor * 255).astype(np.uint8)

def activation_maximization(checkpoint_path, image_path, target_layer):
    model = load_model(checkpoint_path)
    input_tensor = load_image(image_path)
    optimized_tensor = process_activation(model, input_tensor, target_layer)
    return deprocess_image(optimized_tensor)


In [48]:
# for usage
checkpoint_path = 'trained_models/ResNet18_MNIST.pth'
image_path = ''
target_layer = 2
resulting_image = activation_maximization(checkpoint_path, image_path, target_layer)

/home/bini/anaconda3/envs/augcon/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/bini/anaconda3/envs/augcon/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


AttributeError: 'str' object has no attribute 'read'

In [1]:
# generating synthetic dataset using activation maximization for each output class of the given model

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader, TensorDataset

import numpy as np
from PIL import Image

from models import load_model
from utils import load_config


class SyntheticGenerator():
    def __init__(self, pretrained_model, data_config, device, logger, size_per_class):

        self.model = pretrained_model
        self.size = size_per_class
        self.device = device
        self.logger = logger

        self.data_config = data_config

        print(self.data_config["input_shape"])
        self.transform = transforms.Compose([
            transforms.Resize(self.data_config["input_shape"][1:]),
            transforms.ToTensor()
        ])

        self.model.eval()
        self.model.to(self.device)

    def generate_dataset(self, output_dir = None):

        data = []
        data_labels = []
        dataloader = None

        for _ in range(self.size):
            for label in range(self.data_config["num_classes"]):

                gen_img = self.generate_image(label)
                data.append(gen_img)
                data_labels.append(label)

        data = torch.stack(data)
        data_labels = torch.tensor(data_labels)

        dataloader = DataLoader(TensorDataset(data, data_labels), batch_size=32, shuffle=True)

        if output_dir:
            print("Saving dataset to ", output_dir)
            torch.save(dataloader, output_dir)

        return dataloader


    def generate_image(self, label):

        image = self.random_image().to(self.device)
        image.requires_grad = True

        optimizer = optim.Adam([image], lr=0.01)

        for _ in range(1000):

            optimizer.zero_grad()
            output = self.model(image)
            loss = -output[0][label]
            loss.backward()
            optimizer.step()

        return image


    def random_image(self):

        random_high, random_low = 180, 160

        mean=torch.tensor([0.485, 0.456, 0.406]).unsqueeze(0).unsqueeze(2).unsqueeze(3)
        std=torch.tensor([0.229, 0.224, 0.225]).unsqueeze(0).unsqueeze(2).unsqueeze(3)

        random_high, random_low = 180, 160
        
        image = (((random_high - random_low) * torch.rand(self.data_config["input_shape"]) + random_low)/255)
        # image = (image - mean) / std


        return image



In [2]:

config_dir = "configs/model_configs/ResNet18_MNIST.yaml"
model_config = load_config(config_dir)
model, trained = load_model(model_config, torch.device("cuda"))

if not trained:
    raise ValueError("Model must be trained.")

dataset_dir = "configs/dataset_configs/MNIST.yaml"
data_config = load_config(dataset_dir)

device = torch.device("cuda")
logger = None
size_per_class = 1

generator = SyntheticGenerator(model, data_config, device, logger, size_per_class)


print("model loaded")


rand_img = generator.random_image()
mean=torch.tensor([0.485, 0.456, 0.406]).unsqueeze(0).unsqueeze(2).unsqueeze(3)
mean.shape, rand_img.shape

<class 'int'>


/home/bini/anaconda3/envs/augcon/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/bini/anaconda3/envs/augcon/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Model build as resnet18, pretrained: True, output channels: 10
Loading model from ./trained_models/ResNet18_MNIST.pth
Model loaded from ./trained_models/ResNet18_MNIST.pth
[1, 28, 28]
model loaded


(torch.Size([1, 3, 1, 1]), torch.Size([1, 28, 28]))

In [10]:
# res18mnist, trained =
# import os
# print(os.path.curdir)
model_config_dir = "configs/model_configs/ResNet18_MNIST.yaml"
model_config1 = load_config(model_config_dir)


image1 = torch.rand(1, *generator.data_config["input_shape"])

model1, t = load_model(model_config1, torch.device("cuda"))

model1(image1)

# trained

<class 'int'>


/home/bini/anaconda3/envs/augcon/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/bini/anaconda3/envs/augcon/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Model build as resnet18, pretrained: True, output channels: 10
Loading model from ./trained_models/ResNet18_MNIST.pth
Model loaded from ./trained_models/ResNet18_MNIST.pth


ValueError: Expected more than 1 value per channel when training, got input size torch.Size([1, 512, 1, 1])

In [ ]:

model1(image1)

RuntimeError: Input type (torch.FloatTensor) and weight type (torch.cuda.FloatTensor) should be the same or input should be a MKLDNN tensor and weight is a dense tensor